In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
#!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

#!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
print(df_books.shape)
print (df_ratings.shape)

(271379, 3)
(1149780, 3)


In [ ]:
min_user_ratings = 200
min_book_ratings = 100

user_counts = df_ratings['user'].value_counts()
insignificant_users = user_counts[user_counts < min_user_ratings].index
df_ratings_filtered = df_ratings[~df_ratings['user'].isin(insignificant_users)]
print(df_ratings_filtered.shape)

book_counts = df_ratings['isbn'].value_counts()
insignificant_books = book_counts[book_counts < min_book_ratings].index
df_ratings_filtered = df_ratings_filtered[~df_ratings_filtered['isbn'].isin(insignificant_books)]
print(df_ratings_filtered.shape)

df = df_ratings_filtered.pivot_table(index=['user'],columns=['isbn'],values='rating').fillna(0).T

df.index = df.join(df_books.set_index('isbn'))['title']

print(df.head())

model_knn = NearestNeighbors(n_neighbors=5, radius=1.0, algorithm='auto', metric='cosine')
model_knn.fit(df.values)


(527556, 3)
(49781, 3)
user                                                254     2276    2766    \
title                                                                        
Politically Correct Bedtime Stories: Modern Tal...     0.0     0.0     0.0   
Angels                                                 0.0     0.0     0.0   
The Boy Next Door                                      0.0     0.0     0.0   
Men Are from Mars, Women Are from Venus: A Prac...     0.0     0.0     0.0   
Divine Secrets of the Ya-Ya Sisterhood : A Novel       0.0     0.0     0.0   

user                                                2977    3363    4017    \
title                                                                        
Politically Correct Bedtime Stories: Modern Tal...     0.0     0.0     0.0   
Angels                                                 0.0     0.0     0.0   
The Boy Next Door                                      0.0     0.0     0.0   
Men Are from Mars, Women Are from Venus:

NearestNeighbors(metric='cosine')

In [ ]:
def get_recommends(title = ""):
  try:
    book = df.loc[title]
  except KeyError as e:
    print('The given book', e, 'does not exist')
    return

  distance, indice = model_knn.kneighbors([book.values], n_neighbors=6)

  recommended_books = pd.DataFrame({
      'title'   : df.iloc[indice[0]].index.values,
      'distance': distance[0]
    }) \
    .sort_values(by='distance', ascending=False) \
    .head(5).values

  return [title, recommended_books]

In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", array([["I'll Be Seeing You", 0.8016210794448853],
       ['The Weight of Water', 0.7708583474159241],
       ['The Surgeon', 0.7699410915374756],
       ['I Know This Much Is True', 0.7677075266838074],
       ['The Lovely Bones: A Novel', 0.7234864234924316]], dtype=object)]
You passed the challenge! 🎉🎉🎉🎉🎉
